In [44]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import copy
import re
import sys

In [45]:
fake = pd.read_csv("data/raw/Fake.csv")
real = pd.read_csv("data/raw/true.csv")
fake['label'] = 'fake'
real['label'] = 'real'

In [46]:
#Clean text for real data only
real.text = real.text.str.strip()
fake.text = fake.text.str.strip()
real['text'] = real['text'].str.replace('^(.*?)?\s[-]', '', regex=True)
real['text'].head(10)

0     The head of a conservative Republican faction...
1     Transgender people will be allowed for the fi...
2     The special counsel investigation of links be...
3     Trump campaign adviser George Papadopoulos to...
4     President Donald Trump called on the U.S. Pos...
5     The White House said on Friday it was set to ...
6     President Donald Trump said on Thursday he be...
7     While the Fake News loves to talk about my so...
8     Together, we are MAKING AMERICA GREAT AGAIN! ...
9     Alabama Secretary of State John Merrill said ...
Name: text, dtype: object

In [47]:
#real['text'].iloc[1151]

In [48]:
# Combining datasets
all_news = pd.concat([real, fake])
all_news = all_news.replace(r'^\s*$', np.nan, regex=True)
all_news = all_news.dropna(subset=['text'])
all_news.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [49]:
# Cleaning date column
all_news.date = all_news.date.str.strip()

match_condn = r'\b([0-9])\b'
replace_str = r'0\1'
all_news['date'] = all_news['date'].str.replace(match_condn, replace_str, regex=True)

replacement = {
    "^Jan\s": "January ",
    "^Feb\s": "February ",
    "^Mar\s": "March ",
    "^Apr\s": "April ",
    "^Jun\s": "June ",
    "^Jul\s": "July ",
    "^Aug\s": "August ",
    "^Sep\s": "September ",
    "^Oct\s": "October ",
    "^Nov\s": "November ",
    "^Dec\s": "December ",
    "14[-]Feb[-]18$": "February 14, 2018",
    "15[-]Feb[-]18$": "February 15, 2018",
    "16[-]Feb[-]18$": "February 16, 2018",
    "17[-]Feb[-]18$": "February 17, 2018",
    "18[-]Feb[-]18$": "February 18, 2018",
    "19[-]Feb[-]18$": "February 19, 2018",
}

all_news['date'] = all_news['date'].replace(replacement, regex=True)

# Converting date column to datetime
all_news['date'] = pd.to_datetime(all_news['date'], format='%B %d, %Y', errors='coerce')
all_news.isna().sum() #10 columns are empty because they were not dates in the first place

title       0
text        0
subject     0
date       10
label       0
dtype: int64

In [50]:
# Limiting categories of Subjects to only 2 (politicsNews and worldnews)
all_news.query("subject == 'left-news'")

subject_replace = {'News':'other', 
                   'politics':'politicsNews', 
                   'left-news' : 'other',
                   'Government News': 'politicsNews',
                   'US_News': 'politicsNews',
                   'Middle-east': 'worldnews' 
            }

all_news = all_news.replace({"subject": subject_replace})

#Test df to check subjects have been replaced
all_news.query("subject == 'politics'")

,title,text,subject,date,label


In [51]:
#all_copy = copy.deepcopy(all_news)

In [52]:
#all_copy['text'].head(10)

In [53]:
#all_copy.to_csv("all_copy.csv", index=False)

In [54]:
all_news.to_csv("data/processed/all_news.csv", index=False)